In [1]:
import json
import numpy as np
import scipy.stats as sst
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, confusion_matrix, recall_score, precision_recall_fscore_support
import load


def load_dataset(data_json):
    with open(data_json, 'r') as fid:
        data = [json.loads(l) for l in fid]
    labels = []
    for d in data:
        labels.append(d['labels'])
    return labels

Using TensorFlow backend.


In [2]:
%cd ..

/home/q/Documents/master/TEL531E/termproject/ecg-master


In [3]:
classes = ["A", "N", "O", "~"]
classes = sorted(set(classes))

int_to_class = dict(zip(range(len(classes)), classes))
class_to_int = {c : i for i, c in int_to_class.items()}

path_preds = "saved/preds/1607813643-299/preds.txt"
path_preds = "saved/preds/1607813643-299/preds_train.txt"
pred_porbs = np.loadtxt(path_preds)
pred_porbs = pred_porbs.reshape((pred_porbs.shape[0], pred_porbs.shape[1]/len(classes), len(classes)))
y_pred = np.argmax(pred_porbs, axis=2)
#preds = np.array([int_to_class[x] for x in preds[0]])


#path_actual = "examples/cinc17/val.json"
path_actual = "examples/cinc17/train.json"
actual = load.load_dataset(path_actual)

preproc = load.Preproc(*actual)
y_true = preproc.process_y(actual[1])
y_true = np.argmax(y_true, axis=2)

100%|██████████| 7676/7676 [00:16<00:00, 474.04it/s]


In [4]:
def acc(y_true, y_pred):
    pred_values, true_values = [], []
    for i in range(len(y_true)):
        label_len = len(actual[1][i])
        pred_values.append(sst.mode(y_pred[i, :label_len])[0][0])
        true_values.append(sst.mode(y_true[i, :label_len])[0][0])

    return np.array(pred_values), np.array(true_values)

In [5]:
p,t = acc(y_true, y_pred)

In [6]:
print(classification_report(t, p))

              precision    recall  f1-score   support

           0       0.89      0.75      0.81       659
           1       0.95      0.71      0.81      4544
           2       0.57      0.89      0.70      2220
           3       0.73      0.66      0.70       253

   micro avg       0.77      0.77      0.77      7676
   macro avg       0.78      0.75      0.75      7676
weighted avg       0.83      0.77      0.78      7676



In [7]:
scores = precision_recall_fscore_support(
                    t,
                    p,
                    average=None)
print "CINC Average {:3f}".format(np.mean(scores[2][:3]))

CINC Average 0.774257


In [11]:
scores[2][:3]

array([0.81152263, 0.81455458, 0.69669247])